In [1]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask



#Sentences we want sentence embeddings for
sentences = ['치킨은 맛있어',
             '역시 치킨이야. 맛이 없을수가 없지',
             '맛이 없을수 없는 치킨의 그 맛!']

#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/stsb-xlm-r-multilingual")
model = AutoModel.from_pretrained("sentence-transformers/stsb-xlm-r-multilingual")

#Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')

#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [2]:
def sentence_XLM_similarity(text1, text2):
    sentences = [text1, text2]
    #Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')
    
    #Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
        
    #Perform pooling. In this case, mean pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    
    similarity = torch.nn.CosineSimilarity(dim=0)(sentence_embeddings[0], sentence_embeddings[1])
    
    return similarity

## KorNLI, KorSTS Test Dataset Load

In [135]:
korsts_test = pd.read_csv('~/data/KorNLUDatasets/KorSTS/tune_test.tsv', delimiter='\t', header=None, encoding='utf-8')

In [136]:
korsts_test.head(5)

,0,1,2
0,한 소녀가 머리를 스타일링하고 있다.,한 소녀가 머리를 빗고 있다.,2.5
1,한 무리의 남자들이 해변에서 축구를 한다.,한 무리의 소년들이 해변에서 축구를 하고 있다.,3.6
2,한 여성이 다른 여성의 발목을 재고 있다.,한 여자는 다른 여자의 발목을 측정한다.,5.0
3,한 남자가 오이를 자르고 있다.,한 남자가 오이를 자르고 있다.,4.2
4,한 남자가 하프를 연주하고 있다.,한 남자가 키보드를 연주하고 있다.,1.5


## KorSTS Dataset Cosine Similarity 결과

In [121]:
encoded_input1 = tokenizer(text1, padding=True, truncation=True, max_length=128, return_tensors='pt')

In [127]:
text2[989]

nan

In [128]:
text1[989]

'그것은 \'어이없다,\' funny Cide의 트레이너 Barclay tagg는 말했다.\t한편, funny cide의 트레이너인 바클레이 태그는 그 주장을 우스꽝스럽다"라고 불렀다.'

In [129]:
text2[1051]

nan

In [139]:
text1 = []
text2 = []
labels = []
with open('../data/KorNLUDatasets/KorSTS/tune_test.tsv', 'rt', encoding='utf-8') as fIn:
    lines = fIn.readlines()
    for i, line in enumerate(lines):
        s1, s2, score = line.split('\t')
        
        score = score.strip()
        score = float(score) / 5.0
        
        text1.append(s1)
        text2.append(s2)
        labels.append(score)

In [140]:
text1[0:10]

['한 소녀가 머리를 스타일링하고 있다.',
 '한 무리의 남자들이 해변에서 축구를 한다.',
 '한 여성이 다른 여성의 발목을 재고 있다.',
 '한 남자가 오이를 자르고 있다.',
 '한 남자가 하프를 연주하고 있다.',
 '한 여자가 양파를 자르고 있다.',
 '한 남자가 전기 자전거를 타고 있다.',
 '한 남자가 드럼을 치고 있다.',
 '한 남자가 기타를 치고 있다.',
 '한 남자가 기타를 치고 있다.']

In [141]:
text2[0:10]

['한 소녀가 머리를 빗고 있다.',
 '한 무리의 소년들이 해변에서 축구를 하고 있다.',
 '한 여자는 다른 여자의 발목을 측정한다.',
 '한 남자가 오이를 자르고 있다.',
 '한 남자가 키보드를 연주하고 있다.',
 '한 여자가 두부를 자르고 있다.',
 '한 남자가 자전거를 타고 있다.',
 '남자가 기타를 치고 있다.',
 '한 여성이 기타를 연주하고 있다.',
 '한 남자가 트럼펫을 연주하고 있다.']

# Text to vector

In [143]:
# text1 변환
text1_input = tokenizer(text1, padding=True, truncation=True, max_length=128, return_tensors='pt')
with torch.no_grad():
    model_output1 = model(**text1_input)


# text2 변환
text2_input = tokenizer(text2, padding=True, truncation=True, max_length=128, return_tensors='pt')
with torch.no_grad():
    model_output2 = model(**text2_input)



In [148]:
sentence_embeddings_text1 = mean_pooling(model_output1, text1_input['attention_mask'])
sentence_embeddings_text2 = mean_pooling(model_output2, text2_input['attention_mask'])

In [150]:
sentence_embeddings_text1.shape

torch.Size([1379, 768])

In [160]:
# model load
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('stsb-xlm-r-multilingual')

  1%|▏         | 13.1M/1.01G [00:08<08:08, 2.05MB/s]  

KeyboardInterrupt: 

  1%|▏         | 13.1M/1.01G [00:19<08:08, 2.05MB/s]

In [157]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import STSBenchmarkDataReader, InputExample

test_samples = []

with open('../data/KorNLUDatasets/KorSTS/tune_test.tsv', 'rt', encoding='utf-8') as fIn:
    lines = fIn.readlines()
    for line in lines:
        s1, s2, score = line.split('\t')
        score = score.strip()
        score = float(score) / 5.0
        test_samples.append(InputExample(texts= [s1,s2], label=score))
        


In [159]:
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')

In [ ]:
test_evaluator()